In [1]:
import open3d.ml.torch as ml3d
import open3d as o3d


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.12/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.12/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.12/dist-package

In [3]:
import open3d.ml.torch as ml3d
print(dir(ml3d.models))

['KPFCNN', 'PVCNN', 'PointPillars', 'PointRCNN', 'PointTransformer', 'RandLANet', 'SparseConvUnet', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_build_config', '_os']


In [19]:
import torch.nn as nn
import torch.nn.functional as F

In [21]:
!git clone https://github.com/yanx27/Pointnet_Pointnet2_pytorch.git
%cd Pointnet_Pointnet2_pytorch
!pip install torch torchvision

Cloning into 'Pointnet_Pointnet2_pytorch'...
remote: Enumerating objects: 842, done.
remote: Total 842 (delta 0), reused 0 (delta 0), pack-reused 842 (from 1)
Receiving objects: 100% (842/842), 68.77 MiB | 23.53 MiB/s, done.
Resolving deltas: 100% (485/485), done.
Updating files: 100% (59/59), done.
/content/Pointnet_Pointnet2_pytorch


In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from log.classification.pointnet2_ssg_wo_normals.pointnet2_utils import PointNetSetAbstraction

In [23]:
class get_model(nn.Module):
    def __init__(self,num_class,normal_channel=True):
        super(get_model, self).__init__()
        in_channel = 6 if normal_channel else 3
        self.normal_channel = normal_channel
        self.sa1 = PointNetSetAbstraction(npoint=512, radius=0.2, nsample=32, in_channel=in_channel, mlp=[64, 64, 128], group_all=False)
        self.sa2 = PointNetSetAbstraction(npoint=128, radius=0.4, nsample=64, in_channel=128 + 3, mlp=[128, 128, 256], group_all=False)
        self.sa3 = PointNetSetAbstraction(npoint=None, radius=None, nsample=None, in_channel=256 + 3, mlp=[256, 512, 1024], group_all=True)
        self.fc1 = nn.Linear(1024, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.drop1 = nn.Dropout(0.4)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.drop2 = nn.Dropout(0.4)
        self.fc3 = nn.Linear(256, num_class)

    def forward(self, xyz):
        B, _, _ = xyz.shape
        if self.normal_channel:
            norm = xyz[:, 3:, :]
            xyz = xyz[:, :3, :]
        else:
            norm = None
        l1_xyz, l1_points = self.sa1(xyz, norm)
        l2_xyz, l2_points = self.sa2(l1_xyz, l1_points)
        l3_xyz, l3_points = self.sa3(l2_xyz, l2_points)
        x = l3_points.view(B, 1024)
        x = self.drop1(F.relu(self.bn1(self.fc1(x))))
        x = self.drop2(F.relu(self.bn2(self.fc2(x))))
        x = self.fc3(x)
        x = F.log_softmax(x, -1)


        return x, l3_points



class get_loss(nn.Module):
    def __init__(self):
        super(get_loss, self).__init__()

    def forward(self, pred, target, trans_feat):
        total_loss = F.nll_loss(pred, target)

        return total_loss

In [25]:
# Создаём модель
num_classes = 40   # для ModelNet40 (классификация 40 классов)
model = get_model(num_class=num_classes, normal_channel=False)

# Загружаем веса
checkpoint = torch.load("log/classification/pointnet2_ssg_wo_normals/checkpoints/best_model.pth", map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

get_model(
  (sa1): PointNetSetAbstraction(
    (mlp_convs): ModuleList(
      (0): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
    )
    (mlp_bns): ModuleList(
      (0-1): 2 x BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (sa2): PointNetSetAbstraction(
    (mlp_convs): ModuleList(
      (0): Conv2d(131, 128, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
      (2): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1))
    )
    (mlp_bns): ModuleList(
      (0-1): 2 x BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (sa3): PointNetSetAbstract

In [26]:
# Случайное облако точек: batch=1, channels=3, points=1024
xyz = torch.rand(1, 3, 1024)

# Прогон через модель
with torch.no_grad():
    pred, _ = model(xyz)

print("Предсказания:", pred)
print("Класс:", torch.argmax(pred, dim=1))

Предсказания: tensor([[ -5.0069, -13.1186,  -4.6161,  -7.3341,  -3.8622,  -8.3424, -12.0152,
          -4.2633, -11.1426,  -7.5980, -12.4904,  -2.4201,  -6.0125,  -9.6002,
          -5.9130,  -4.4373, -10.7899,  -9.6642,  -6.4548,  -9.0622,  -9.7181,
          -8.9397,  -7.5343,  -9.6315,  -4.3642,  -6.5884,  -0.3091,  -8.0309,
          -6.9950, -10.8308,  -5.9543,  -2.7975, -12.0323,  -8.0475,  -7.6652,
         -11.3940,  -5.1220,  -8.4801,  -4.1127,  -6.2190]])
Класс: tensor([26])


26 - это тумбочка